## Introduction

The patternmind_dataset contains thousands of labeled images across 233 diverse categories, each represented by extracted visual features.

This is primarily a clustering problem combined with exploratory analysis. While the dataset has ground-truth labels (making supervised classification possible), our main goal is to understand how images naturally group together based on their visual features.

Our approach follows these steps:

1. Data Loading: Load image paths and labels, check for missing values and duplicates
2. Exploratory Data Analysis (EDA): Understand dataset characteristics, class distribution and image properties
3. Feature Extraction: Use a Convolutional Neural Network (CNN) to extract meaningful 256-dimensional feature vectors from each image
4. Dimensionality Reduction: Apply PCA to reduce features to 60 dimensions while preserving ~87% of variance
5. Unsupervised Clustering: Test two different clustering algorithms (K-Means, DBSCAN)
6. Supervised Baseline: Train a Logistic Regression classifier as a reference for comparison

## 1. Data Loading and Initial Exploration

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import keras
from keras import layers
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### 1.1 Load Image Paths and Labels

We load all images from the dataset directory, where each subfolder represents a distinct category. The code below creates a DataFrame containing the file path, category name, and numeric label for each image.

In [2]:
dataset_path = "db/patternmind_dataset/"

# Get all category folders (each subfolder is a class)
categories = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(
    os.path.join(dataset_path, d)) and not d.startswith('.')])

# Build a list of all images with their metadata
data = []
for label_id, category in enumerate(categories):
    folder_path = os.path.join(dataset_path, category)
    images = [f for f in os.listdir(
        folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img in images:
        data.append({
            'path': os.path.join(folder_path, img),
            'category': category,
            'label_id': label_id
        })

df = pd.DataFrame(data)

### 1.2 Dataset Statistics and Quality Check

The code below checks for missing values, duplicates and computes summary statistics about the class distribution.

In [3]:
# Check for missing values and data quality
print(f"Missing values in DataFrame: {df.isnull().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

# Detailed statistics
counts = df['category'].value_counts()
print(f"\nTotal images: {len(df)}")
print(f"Total categories: {len(categories)}")
print(f"Average images per category: {len(df) / len(categories):.2f}")
print(f"Min images per category: {counts.min()}")
print(f"Max images per category: {counts.max()}")
print(f"Median images per category: {counts.median()}")
print(f"Std deviation: {counts.std():.2f}")
print("\nEdge cases (fewest images):")
print(counts.tail())
print("\nEdge cases (most images):")
print(counts.head())

# Class imbalance analysis
imbalance_ratio = counts.max() / counts.min()
print(f"\nClass imbalance ratio (max/min): {imbalance_ratio:.2f}")

Missing values in DataFrame: 0
Duplicate rows: 0

Total images: 25557
Total categories: 233
Average images per category: 109.69
Min images per category: 72
Max images per category: 761
Median images per category: 92.0
Std deviation: 81.32

Edge cases (fewest images):
category
scorpion              73
golden-gate-bridge    73
harpsichord           73
sunflower             73
top-hat               72
Name: count, dtype: int64

Edge cases (most images):
category
clutter       761
airplanes     720
motorbikes    719
faces-easy    396
t-shirt       330
Name: count, dtype: int64

Class imbalance ratio (max/min): 10.57


The output above shows:

- No missing values or duplicates
- 25557 total images across 233 categories
- Average of ~110 images per category, but with significant variation
- Class imbalance ratio of 10.57x between the most and least frequent categories

This imbalance is important to consider: rare categories may be underrepresented in clusters, and supervised classifiers might favor frequent categories. We will address this using stratified sampling and evaluation metrics that account for class imbalance.

## 2. Exploratory Data Analysis (EDA)

### 2.1 Class Distribution Analysis

We first examine how images are distributed across categories.

In [4]:
fig_dist = px.bar(counts, x=counts.index, y=counts.values,
                  title="Class Distribution",
                  labels={'index': 'Category', 'y': 'Count'})
fig_dist.update_layout(xaxis={'categoryorder': 'total descending'})
fig_dist.show()

![Class Distribution](images/2-1.png)

The bar chart reveals significant class imbalance in the dataset:

- The most frequent category ("clutter") has 761 images
- The least frequent category ("top-hat") has only 72 images
- This represents a 10x variation in class sizes

Rare categories may not form distinct clusters and could be absorbed into larger groups. The model might achieve high accuracy by simply predicting frequent classes. We use stratified train-test splitting and report both macro-averaged (treats all classes equally) and weighted metrics (reflects real-world distribution)

### 2.2 Image Dimension Analysis

Before feeding images to a neural network, we need to understand the variation in image sizes. Neural networks require fixed-size inputs, so we must choose an appropriate target resolution. We analyze a sample of 5,000 images to understand the distribution of widths and heights.

In [5]:
# Sample images to analyze dimension variability
sample_size = 5000
sample_df = df.sample(n=sample_size)
sizes = [Image.open(p).size for p in sample_df['path']]
widths, heights = zip(*sizes)

# Create side-by-side histograms for width and height distributions
fig_sizes = make_subplots(rows=1, cols=2, subplot_titles=(
    "Width Distribution", "Height Distribution"))
fig_sizes.add_trace(go.Histogram(x=widths, name="Width"), row=1, col=1)
fig_sizes.add_trace(go.Histogram(x=heights, name="Height"), row=1, col=2)
fig_sizes.update_layout(
    title_text=f"Image Size Distribution (Sample n={sample_size})")
fig_sizes.show()

![Image Dimension Distribution](images/2-2.png)

The histograms show that image dimensions vary considerably:

- Most images have widths and heights concentrated around 200-400 pixels
- Some images are very small (thumbnails) while others are much larger

We chose 224×224 resolution (the ImageNet standard) because it is large enough to preserve important visual details like edges, textures, and shapes, while remaining computationally efficient by reducing memory usage and training time. This resolution is also an industry standard for image classification tasks.

### 2.3 Visual Sample Inspection

To understand the visual diversity in our dataset, we display random samples from five different categories. This helps us appreciate the challenges our models will face: varying backgrounds, lighting conditions, object orientations and image quality.

In [6]:
sample_categories = np.random.choice(categories, 5, replace=False)
fig_imgs = make_subplots(rows=1, cols=5, subplot_titles=sample_categories)

for i, cat in enumerate(sample_categories):
    img_path = df[df['category'] == cat].sample(1).iloc[0]['path']
    img = Image.open(img_path)
    img_array = np.array(img)
    fig_imgs.add_trace(go.Image(z=img_array), row=1, col=i+1)

fig_imgs.update_layout(title_text="Sample Images from Random Categories")
fig_imgs.show()

![Sample Images](images/2-3.png)

The random samples reveal substantial visual diversity within and across categories.

To address this variability, we apply data augmentation techniques such as random flips, rotations, and zoom, which help the model generalize to different orientations. Additionally, our CNN architecture with multiple convolutional layers learns hierarchical features that are robust to these variations.

## 3. CNN Feature Extraction

We use a CNN for feature extraction because these networks learn representations at multiple levels of complexity. Early layers detect simple patterns like edges and textures, while deeper layers recognize more complex features like shapes and objects. The layer just before the final classification output contains a rich, compressed representation of the image that captures its essential visual characteristics. These learned features work much better for clustering than using raw pixel values directly.

### 3.1 Data Preprocessing and Augmentation

Before training the CNN, we prepare the data with preprocessing and augmentation techniques.

- Resize to 224×224: Standardizes all images to the same dimensions
- Normalize to [0, 1]: Divides pixel values by 255 to scale them between 0 and 1, which helps neural network training converge faster
- Random Horizontal Flip: Creates mirror images, teaching the model that objects can face either direction
- Random Rotation (±10%): Handles slight orientation differences in photographs
- Random Zoom (±10%): Accounts for objects appearing at different scales

We split the data using 80% for training and 20% for validation (to monitor overfitting)

In [7]:
# type: ignore
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation: random flips, rotations, and zoom
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Load training and validation datasets (80/20 split)
train_ds = keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Apply augmentation to training data and normalize both datasets
train_ds = train_ds.map(lambda x, y: (
    data_augmentation(x, training=True), y))
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

# Optimize pipeline with caching, shuffling, and prefetching
train_ds = train_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(tf.data.AUTOTUNE)

Found 25557 files belonging to 233 classes.
Using 20446 files for training.
Found 25557 files belonging to 233 classes.
Using 5111 files for validation.


### 3.2 CNN Model Architecture

We design a custom CNN with three convolutional blocks to extract visual features from images.

| Layer Type   | Details                 | Purpose                                        |
| ------------ | ----------------------- | ---------------------------------------------- |
| Conv Block 1 | 32 filters, 3×3 kernel  | Detect low-level features (edges, colors)      |
| Conv Block 2 | 64 filters, 3×3 kernel  | Detect mid-level features (textures, patterns) |
| Conv Block 3 | 128 filters, 3×3 kernel | Detect high-level features (shapes, parts)     |
| Dense Layer  | 256 neurons             | Compress features into final representation    |
| Output Layer | 233 neurons, softmax    | Classify into one of 233 categories            |

Key components in each block:

- Conv2D: Applies learnable filters to detect features
- BatchNormalization: Normalizes activations, stabilizing and speeding up training
- MaxPooling: Reduces spatial dimensions by half, providing translation invariance
- Dropout (25%): Randomly deactivates neurons to prevent overfitting

Training configuration:

- Optimizer: Adam with learning rate 0.001 (adapts learning rate per parameter)
- Loss function: Categorical crossentropy (standard for multi-class classification)
- Early stopping: Stops training if validation loss doesn't improve for 5 epochs
- Learning rate reduction: Halves learning rate if validation loss plateaus

Our goal isn't to achieve the best classification accuracy: it's to extract good features. The 256-dimensional representation from the Dense layer is what we use for clustering.

In [8]:
MODEL_PATH = 'out/models/cnn_feature_extractor.keras'

if os.path.exists(MODEL_PATH):
    model = keras.models.load_model(MODEL_PATH)
    print("Model loaded successfully!")
else:
    # Build CNN with 3 convolutional blocks (32, 64, 128 filters)
    model = keras.Sequential([
        layers.Input(shape=(224, 224, 3)),

        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Dense layers for classification
        layers.Flatten(),
        layers.Dense(256, activation='relu', name='features'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(len(categories), activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Training callbacks for early stopping and learning rate reduction
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7
        )
    ]

    history = model.fit(
        train_ds,
        epochs=30,
        validation_data=val_ds,
        callbacks=callbacks
    )

    # Save the trained model
    os.makedirs('out/models', exist_ok=True)
    model.save(MODEL_PATH)
    print(f"Model saved to {MODEL_PATH}")

Model loaded successfully!


| Epoch | Accuracy   | Loss      | Val Accuracy | Val Loss  | Learning Rate |
| ----- | ---------- | --------- | ------------ | --------- | ------------- |
| 1     | 9.36%      | 4.967     | 11.78%       | 4.734     | 0.001         |
| 2     | 15.69%     | 4.291     | 16.57%       | 4.252     | 0.001         |
| 3     | 20.72%     | 3.900     | 19.15%       | 4.096     | 0.001         |
| 4     | 25.54%     | 3.536     | 21.58%       | 3.860     | 0.001         |
| 5     | 32.08%     | 3.115     | 24.09%       | 3.785     | 0.001         |
| **6** | **41.81%** | **2.558** | **25.04%**   | **3.747** | **0.001**     |
| 7     | 55.40%     | 1.871     | 23.79%       | 4.034     | 0.001         |
| 8     | 70.89%     | 1.180     | 23.48%       | 4.199     | 0.001         |
| 9     | 82.04%     | 0.710     | 23.46%       | 4.211     | 0.001         |
| 10    | 90.88%     | 0.375     | 25.89%       | 4.221     | 0.0005        |
| 11    | 94.25%     | 0.250     | 25.96%       | 4.265     | 0.0005        |

The CNN trained for 11 epochs before early stopping triggered, restoring the best weights from epoch 6.

| Metric                   | Value    | Interpretation                                     |
| ------------------------ | -------- | -------------------------------------------------- |
| Final training accuracy  | ~94%     | Model learned training data patterns well          |
| Best validation accuracy | ~25%     | Achieved at epoch 6 (restored weights)             |
| Best validation loss     | 3.747    | Epoch 6, lowest during training                    |
| Early stopping           | Epoch 11 | Validation loss increased for 5 consecutive epochs |

Training progression:

- Epochs 1-6: Steady improvement in both training and validation metrics
- Epochs 7-11: Training accuracy continued rising (42% → 94%) while validation loss increased (3.75 → 4.27), confirming overfitting

The validation accuracy gap is primarily explained by the nature of this dataset: with 233 classes and only ~88 images per class after the 80/20 split, the model has limited examples to learn from. Additionally, many categories share visual similarities (e.g., different animals, different vehicles), making fine-grained distinctions difficult. To put 25% in perspective, random guessing would achieve only ~0.43% accuracy (1/233 classes), meaning our model performs approximately 58× better than random, clear evidence that it learned meaningful visual patterns. Most importantly, for our feature extraction purposes, the exact classification accuracy matters less than the quality of the learned representations. The CNN's intermediate layers capture rich visual information that will serve as the foundation for our clustering analysis.

### 3.3 Feature Extraction from All Images

Now we extract 256-dimensional feature vectors from every image in the dataset. We do this by:

1. Removing the final classification layer from the trained CNN
2. Passing each image through the network
3. Collecting the output from the Dense(256) layer

These features capture the "essence" of each image as learned by the CNN: a compact representation that preserves visual similarity. Images that look alike will have similar feature vectors making them suitable for clustering.

In [9]:
FEATURES_PATH = 'out/features/features.npy'
LABELS_PATH = 'out/features/labels.npy'

if os.path.exists(FEATURES_PATH) and os.path.exists(LABELS_PATH):
    features = np.load(FEATURES_PATH)
    labels = np.load(LABELS_PATH)
    print(f"Features loaded: {features.shape}")
    print(f"Labels loaded: {labels.shape}")
else:
    # Create feature extractor by removing final classification layer
    feature_extractor = keras.Sequential(model.layers[:-1])  # type: ignore

    # Load all images without augmentation
    all_ds = keras.utils.image_dataset_from_directory(
        dataset_path,
        label_mode='int',
        shuffle=False,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )

    all_ds = all_ds.map(lambda x, y: (x / 255.0, y))  # type: ignore

    # Extract 256-dimensional features for all images
    features = feature_extractor.predict(all_ds.map(lambda x, y: x))
    labels = np.concatenate([y.numpy() for _, y in all_ds], axis=0)

    # Save features and labels
    os.makedirs('out/features', exist_ok=True)
    np.save(FEATURES_PATH, features)
    np.save(LABELS_PATH, labels)
    print(f"Features extracted and saved:")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}")

Features loaded: (25557, 256)
Labels loaded: (25557,)


## 4. Dimensionality Reduction with PCA

256 dimensions can be challenging for some clustering algorithms. Principal Component Analysis (PCA) further reduces dimensionality while preserving the most important information (variance) in the data.

### 4.1 Feature Standardization and Variance Analysis

1. We standardize all 256 features
2. We fit PCA to see how variance is distributed across components
3. We choose the optimal number of components to retain

In [10]:
features_loaded = np.load(FEATURES_PATH)
labels_loaded = np.load(LABELS_PATH)

# Standardize features (mean=0, std=1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_loaded)

# Fit PCA to analyze variance distribution
pca_full = PCA()
pca_full.fit(features_scaled)

explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

fig_variance = go.Figure()
fig_variance.add_trace(go.Scatter(
    x=list(range(1, len(explained_variance) + 1)),
    y=cumulative_variance,
    mode='lines+markers',
    name='Cumulative Variance'
))
fig_variance.update_layout(
    title='PCA Explained Variance',
    xaxis_title='Number of Components',
    yaxis_title='Cumulative Explained Variance',
    hovermode='x'
)
fig_variance.show()

print(
    f"Variance explained by first 2 components: {cumulative_variance[1]:.2%}")
print(
    f"Variance explained by first 10 components: {cumulative_variance[9]:.2%}")
print(
    f"Variance explained by first 50 components: {cumulative_variance[49]:.2%}")
print(
    f"Variance explained by first 60 components: {cumulative_variance[59]:.2%}")
print(
    f"Variance explained by first 75 components: {cumulative_variance[74]:.2%}")
print(
    f"Variance explained by first 100 components: {cumulative_variance[99]:.2%}")
print(
    f"Variance explained by first 120 components: {cumulative_variance[149]:.2%}")

Variance explained by first 2 components: 14.36%
Variance explained by first 10 components: 42.61%
Variance explained by first 50 components: 82.91%
Variance explained by first 60 components: 87.14%
Variance explained by first 75 components: 92.12%
Variance explained by first 100 components: 97.93%
Variance explained by first 120 components: 100.00%


![PCA Explained Variance](images/4-1.png)

The plot shows how much information (variance) is retained as we add more principal components:

| Components | Variance Explained |
| ---------- | ------------------ |
| First 2    | 14.36%             |
| First 10   | 42.61%             |
| First 50   | 82.91%             |
| First 60   | 87.14%             |
| First 75   | 92.12%             |
| First 100  | 97.93%             |
| First 120  | 100.00%            |

The variance is distributed across approximately 120 effective dimensions (not all 256), indicating that the CNN's 256 dimensional output contains significant redundancy: over half of the original dimensions contribute no additional information. Notably:

- The first 50 components capture the majority of useful information (~83%)
- Beyond 100 components, we're capturing only noise and minor variations
- The curve shows gradual accumulation without a sharp "elbow," meaning many features contribute meaningful information

Our choice is 60 components: This configuration retains approximately 87% of the total variance while reducing dimensionality by over 75% compared to the original 256 dimensions.

### 4.2 2D Visualization

While 60 dimensions is optimal for clustering, we cannot visualize 60D data directly. We project features to just 2 dimensions (PC1 and PC2) to create scatter plots that give us intuition about the data structure. To reduce visual clutter, we focus on only the top 10 most frequent categories. Note that this 2D projection captures only ~14% of the total variance, so clusters that appear overlapping here may be well-separated in higher dimensions.

In [11]:
# Reduce to 60D for clustering
pca_60d = PCA(n_components=60)
features_60d = pca_60d.fit_transform(features_scaled)

# Reduce to 2D for visualization
pca_2d = PCA(n_components=2)
features_2d = pca_2d.fit_transform(features_scaled)

# Visualize top 10 most frequent categories
top_categories = df['category'].value_counts().head(10).index.tolist()
top_category_indices = [i for i, cat in enumerate(
    categories) if cat in top_categories]

mask = np.isin(labels_loaded, top_category_indices)
df_pca_top = pd.DataFrame({
    'PC1': features_2d[mask, 0],
    'PC2': features_2d[mask, 1],
    'category': [categories[label] for label in labels_loaded[mask]]
})

fig_pca_top = px.scatter(
    df_pca_top,
    x='PC1',
    y='PC2',
    color='category',
    title='PCA 2D Projection - Top 10 Categories',
    opacity=0.7
)
fig_pca_top.update_traces(marker=dict(size=4))
fig_pca_top.update_layout(width=1000, height=800)
fig_pca_top.show()

![PCA 2D Projection - Top 10 Categories](images/4-2.png)

The scatter plot reveals a mix of separation and overlap among the top 10 categories. Motorbikes (pink) form a distinct cluster in the lower-right region, while faces-easy (orange) concentrates in the upper portion: both categories have unique visual signatures that make them easier to distinguish.

In contrast, categories like airplanes and bathtubs heavily overlap in the central region, indicating they share similar visual features such as edges, textures, or color patterns. Categories like hammock and billiards are scattered throughout without forming tight groups, reflecting high variability within those classes.

The key takeaway is that while some categories naturally separate even in 2D, most overlap significantly. This confirms that the additional 58 dimensions (beyond these first 2) are essential for distinguishing between visually similar categories during clustering.

## 5. Unsupervised Clustering

We apply two clustering algorithms to find natural groupings in the 60D feature space. Since these methods don't use category labels, we can evaluate how well visual similarity alone groups images.

Algorithms:

- K-Means: Partitions data into k clusters by minimizing distance to cluster centers. We specify k beforehand.
- DBSCAN: Finds dense regions automatically and labels sparse points as noise. No need to specify cluster count.

### 5.1 K-Means Clustering

We evaluate K-Means across multiple values of k (50, 100, 150, and 233) to understand how cluster count affects performance. For each configuration we measure:

- Silhouette score: Cluster cohesion and separation (range -1 to 1, higher is better)
- Purity: Fraction of each cluster belonging to the dominant ground-truth category
- Inertia: Sum of squared distances to cluster centers (lower indicates tighter clusters)

In [12]:
k_values = [50, 100, 150, 233]
kmeans_results = {}

for k in k_values:
    print(f"\nFitting K-Means with k={k}...")

    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_60d)

    # Silhouette score: measures cluster separation [-1, 1], higher is better
    silhouette_avg = silhouette_score(
        features_60d, cluster_labels, sample_size=5000)

    # Cluster purity: fraction of dominant category in each cluster
    purity_scores = []
    for cluster_id in range(k):
        cluster_mask = cluster_labels == cluster_id
        if cluster_mask.sum() > 0:
            cluster_true_labels = labels_loaded[cluster_mask]
            most_common_label = np.bincount(cluster_true_labels).argmax()
            purity = (cluster_true_labels ==
                      most_common_label).sum() / cluster_mask.sum()
            purity_scores.append(purity)

    avg_purity = np.mean(purity_scores)

    kmeans_results[k] = {
        'model': kmeans,
        'labels': cluster_labels,
        'silhouette': silhouette_avg,
        'purity': avg_purity,
        'inertia': kmeans.inertia_
    }

    print(f"K={k}: Silhouette={silhouette_avg:.3f}, Purity={
          avg_purity:.3f}, Inertia={kmeans.inertia_:.0f}")


Fitting K-Means with k=50...
K=50: Silhouette=0.029, Purity=0.242, Inertia=1609672

Fitting K-Means with k=100...
K=100: Silhouette=0.019, Purity=0.250, Inertia=1494335

Fitting K-Means with k=150...
K=150: Silhouette=0.018, Purity=0.249, Inertia=1433556

Fitting K-Means with k=233...
K=233: Silhouette=0.011, Purity=0.253, Inertia=1373468


| k   | Silhouette | Purity | Inertia   |
| --- | ---------- | ------ | --------- |
| 50  | 0.030      | 24.2%  | 1,609,672 |
| 100 | 0.020      | 25.0%  | 1,494,532 |
| 150 | 0.017      | 24.9%  | 1,433,556 |
| 233 | 0.011      | 25.3%  | 1,373,468 |

1. Silhouette decreases with k: More clusters create less cohesive groups (0.030 → 0.011).
2. Purity plateaus at ~25%: Even matching the true 233 categories, only 1 in 4 images lands in a cluster dominated by its correct category.

K-Means struggles because the 233 categories don't form distinct spherical clusters as many share visual features. Best choice is k=233 for highest purity, though the marginal gains over k=50 are minimal.

In [13]:
fig_kmeans_metrics = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Silhouette Score', 'Average Purity', 'Inertia')
)

k_list = list(kmeans_results.keys())
silhouette_scores = [kmeans_results[k]['silhouette'] for k in k_list]
purity_scores = [kmeans_results[k]['purity'] for k in k_list]
inertia_scores = [kmeans_results[k]['inertia'] for k in k_list]

fig_kmeans_metrics.add_trace(
    go.Scatter(x=k_list, y=silhouette_scores,
               mode='lines+markers', name='Silhouette'),
    row=1, col=1
)
fig_kmeans_metrics.add_trace(
    go.Scatter(x=k_list, y=purity_scores, mode='lines+markers', name='Purity'),
    row=1, col=2
)
fig_kmeans_metrics.add_trace(
    go.Scatter(x=k_list, y=inertia_scores,
               mode='lines+markers', name='Inertia'),
    row=1, col=3
)

fig_kmeans_metrics.update_xaxes(title_text="Number of Clusters (k)")
fig_kmeans_metrics.update_layout(
    height=400, showlegend=False, title_text="K-Means Evaluation Metrics")
fig_kmeans_metrics.show()

![K-Means Evaluation Metrics](images/5-1.png)

- Left (Silhouette): Drops from 0.030 to 0.011 as k increases, more clusters means less cohesive groups.
- Center (Purity): Rises slightly then plateaus at ~25%. Even with k=233, only 1 in 4 images ends up in a cluster dominated by its true category.
- Right (Inertia): Smooth decline with no "elbow" point, suggesting the data has no natural cluster count.

These metrics conflict: silhouette favors fewer clusters (k=50) while purity slightly favors more (k=233). The ~25% purity ceiling shows that visual features alone don't cleanly separate the 233 categories, as many share similar shapes, textures, and colors.

### 5.2 DBSCAN Density-Based Clustering

Unlike K-Means, DBSCAN doesn't require specifying the number of clusters beforehand. Instead, it finds dense regions in the feature space and labels sparse points as noise. This makes it useful for discovering the "natural" structure of our data without forcing it into a predetermined number of groups.

Key parameters:

- eps: Maximum distance between two points to be considered neighbors
- min_samples: Minimum points required to form a dense region (cluster core)

We first use a k-distance graph to identify an appropriate eps value, then apply DBSCAN to our 60D features.

In [14]:
# Sample data for k-distance computation
sample_size_dbscan = 10000
sample_indices_nn = np.random.choice(
    len(features_60d), size=sample_size_dbscan, replace=False)
features_sample = features_60d[sample_indices_nn]

# Find 5 nearest neighbors for each point
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(features_sample)
distances, indices = neighbors_fit.kneighbors(features_sample)

# Plot distance to 5th nearest neighbor (elbow suggests good eps value)
distances_sorted = np.sort(distances[:, 4], axis=0)

fig_nn = go.Figure()
fig_nn.add_trace(go.Scatter(
    x=list(range(len(distances_sorted))),
    y=distances_sorted,
    mode='lines',
    name='5th Nearest Neighbor Distance'
))
fig_nn.update_layout(
    title='K-Distance Graph for DBSCAN Parameter Selection',
    xaxis_title='Points sorted by distance',
    yaxis_title='5th Nearest Neighbor Distance',
    height=500
)
fig_nn.show()

print(
    f"Suggested eps values (elbow points): {np.percentile(distances_sorted, [50, 75, 90])}")

Suggested eps values (elbow points): [8.0303998  8.92828846 9.89994516]


![K-Distance Graph](images/5-2-1.png)

- Flat region (0-9k points): Most points have neighbors within distance ~5-10, indicating relatively uniform density throughout the bulk of the dataset.
- Elbow/spike (rightmost ~500 points): A sharp increase where distances jump to 20-160, marking the boundary between dense regions and outliers.

Parameter selection:

- eps = 8.03: Chosen before the elbow point, where the curve starts rising sharply. This captures the natural neighbor distance for the dense majority while excluding extreme outliers.
- min_samples = 10: Conservative choice based on the 2× dimensionality rule of thumb for 60D features.

The elbow occurs very late in the curve (around the 95th percentile), meaning most data points have similar neighbor distances. This indicates the dataset forms one large dense region rather than multiple distinct clusters.

We visualize DBSCAN results by projecting cluster assignments onto the 2D PCA space. A random sample of 5,000 points is used to reduce visual clutter while preserving the overall distribution pattern.

In [15]:
# DBSCAN finds dense regions and labels outliers as noise (-1)
eps_value = 8.02680206
min_samples_value = 10

print(
    f"\nFitting DBSCAN with eps={eps_value}, min_samples={min_samples_value}...")
dbscan = DBSCAN(eps=eps_value, min_samples=min_samples_value)
dbscan_labels = dbscan.fit_predict(features_60d)

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)
noise_percentage = (n_noise / len(dbscan_labels)) * 100

print(f"Number of clusters: {n_clusters_dbscan}")
print(f"Number of noise points: {n_noise} ({noise_percentage:.2f}%)")

non_noise_mask = dbscan_labels != -1
if non_noise_mask.sum() > 0:
    dbscan_silhouette = silhouette_score(
        features_60d[non_noise_mask], dbscan_labels[non_noise_mask], sample_size=5000)
    print(f"Silhouette score (excluding noise): {dbscan_silhouette:.3f}")

sample_indices_dbscan = np.random.choice(
    len(features_2d), size=5000, replace=False)

df_dbscan = pd.DataFrame({
    'PC1': features_2d[sample_indices_dbscan, 0],
    'PC2': features_2d[sample_indices_dbscan, 1],
    'cluster': dbscan_labels[sample_indices_dbscan].astype(str),
    'is_noise': dbscan_labels[sample_indices_dbscan] == -1,
    'category': [categories[label] for label in labels_loaded[sample_indices_dbscan]]
})

df_dbscan['cluster'] = df_dbscan['cluster'].replace('-1', 'Noise')

fig_dbscan = px.scatter(
    df_dbscan,
    x='PC1',
    y='PC2',
    color='cluster',
    hover_data=['category'],
    title=f'DBSCAN Clustering (eps={eps_value}, min_samples={
        min_samples_value}) - PCA 2D',
    opacity=0.6
)
fig_dbscan.update_traces(marker=dict(size=3))
fig_dbscan.update_layout(width=1000, height=800, showlegend=False)
fig_dbscan.show()


Fitting DBSCAN with eps=8.02680206, min_samples=10...
Number of clusters: 2
Number of noise points: 7606 (29.76%)
Silhouette score (excluding noise): 0.153


![DBSCAN Clustering Results](images/5-2-2.png)

The scatter plot visualizes DBSCAN clustering results projected onto the 2D PCA space. With `eps=8.02680206` and `min_samples=10`:

| Metric           | Value          |
| ---------------- | -------------- |
| Clusters found   | 2              |
| Noise points     | 7,606 (29.76%) |
| Silhouette score | 0.178          |

DBSCAN identified only 2 clusters despite 233 ground-truth categories, confirming that the 60D feature space has relatively uniform density without distinct dense regions. Nearly 30% of points were labeled as noice

The silhouette score of 0.178 (higher than K-Means' 0.011-0.030 range) suggests the two clusters DBSCAN found are more cohesive than K-Means forced partitions. However, the algorithm's inability to find more structure reflects the fundamental challenge: visually similar categories blend together in feature space rather than forming distinct dense regions.

DBSCAN reveals that the data's "natural" structure consists of one large dense blob with some outliers, not 233 separate clusters. This validates our earlier observation that visual features alone cannot cleanly separate all semantic categories.

## 6. Supervised Baseline Classifier

To put our clustering results in perspective, we train a supervised classifier that uses the ground-truth labels. This serves as an upper bound: if a simple classifier with access to labels struggles, we shouldn't expect unsupervised clustering (without labels) to do better.

We use Logistic Regression because it's fast, interpretable, and works well with the 256-dimensional CNN features. The classifier learns decision boundaries between categories, giving us a reference point for how separable these classes actually are.


### 6.1 Train-Test Split with Stratification

We split our data into 80% training and 20% test sets using stratified sampling. Stratification ensures that each category maintains the same proportion in both sets.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled,
    labels_loaded,
    test_size=0.2,
    random_state=42,
    stratify=labels_loaded
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Feature dimensions: {X_train.shape[1]}")
print(f"Number of classes: {len(np.unique(y_train))}")

Training set size: 20445
Test set size: 5112
Feature dimensions: 256
Number of classes: 233


### 6.2 Train Logistic Regression Classifier

We train a multinomial Logistic Regression model on the 256-dimensional CNN features. Key settings:

- max_iter=1000: Allows enough iterations for the optimizer to converge with 233 classes
- solver='lbfgs': Efficient optimization algorithm for multinomial classification
- n_jobs=-1: Uses all CPU cores to speed up training

The model learns a separate set of weights for each of the 233 categories, finding linear decision boundaries in the feature space.

In [17]:
lr_classifier = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs',
    verbose=1,
    n_jobs=-1
)

lr_classifier.fit(X_train, y_train)

y_train_pred = lr_classifier.predict(X_train)
y_test_pred = lr_classifier.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Macro-averaged: treat all classes equally
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_test, y_test_pred, average='macro', zero_division=0
)

# Weighted-averaged: weight by class frequency
precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
    y_test, y_test_pred, average='weighted', zero_division=0
)

print(f"\nTraining Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"\nMacro-averaged metrics:")
print(f"  Precision: {precision_macro:.4f}")
print(f"  Recall: {recall_macro:.4f}")
print(f"  F1-Score: {f1_macro:.4f}")
print(f"\nWeighted-averaged metrics:")
print(f"  Precision: {precision_weighted:.4f}")
print(f"  Recall: {recall_weighted:.4f}")
print(f"  F1-Score: {f1_weighted:.4f}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.



Training Accuracy: 0.4992
Test Accuracy: 0.3110

Macro-averaged metrics:
  Precision: 0.2460
  Recall: 0.2532
  F1-Score: 0.2443

Weighted-averaged metrics:
  Precision: 0.2980
  Recall: 0.3110
  F1-Score: 0.2995


| Metric          | Training | Test   | Interpretation                                                                            |
| --------------- | -------- | ------ | ----------------------------------------------------------------------------------------- |
| Accuracy        | 49.92%   | 31.10% | ~18% gap indicates some overfitting; test accuracy is 72× better than random (0.43%)      |
| Macro Precision | -        | 24.60% | Model struggles equally across all categories, including rare ones                        |
| Macro Recall    | -        | 25.32% | On average, ~1 in 4 images per category is correctly identified                           |
| Macro F1        | -        | 24.43% | Balanced performance across classes, no strong bias toward frequent categories            |
| Weighted F1     | -        | 29.95% | Slightly higher: model performs better on frequent categories with more training examples |

- The supervised classifier with full label access achieves only ~31% test accuracy, confirming that these 233 categories are inherently difficult to separate based on visual features alone
- This validates our clustering results: if a model with labels struggles, unsupervised methods without labels cannot be expected to do better
- The small gap between macro and weighted metrics suggests the model doesn't heavily favor frequent classes, it struggles uniformly across categories

### 6.3 Per-Category Performance Analysis

We examine how classification accuracy varies across individual categories. This reveals which types of images are easiest or hardest to classify, and whether performance correlates with category size or visual distinctiveness.

In [18]:
# Calculate per-category accuracy
per_class_accuracy = []
for i, category in enumerate(categories):
    mask = y_test == i
    if mask.sum() > 0:
        class_acc = (y_test_pred[mask] == i).sum() / mask.sum()
        per_class_accuracy.append({
            'category': category,
            'label_id': i,
            'accuracy': class_acc,
            'test_samples': mask.sum()
        })

df_class_acc = pd.DataFrame(per_class_accuracy)
df_class_acc_sorted = df_class_acc.sort_values('accuracy', ascending=False)

print("\nTop 10 Best Performing Categories:")
print(df_class_acc_sorted.head(10)[
      ['category', 'accuracy', 'test_samples']].to_string(index=False))

print("\nTop 10 Worst Performing Categories:")
print(df_class_acc_sorted.tail(10)[
      ['category', 'accuracy', 'test_samples']].to_string(index=False))

print(f"\nMean per-class accuracy: {df_class_acc['accuracy'].mean():.4f}")
print(f"Median per-class accuracy: {df_class_acc['accuracy'].median():.4f}")
print(f"Std per-class accuracy: {df_class_acc['accuracy'].std():.4f}")

fig_acc_dist = go.Figure()
fig_acc_dist.add_trace(go.Histogram(
    x=df_class_acc['accuracy'],
    nbinsx=30,
    name='Per-Class Accuracy'
))
fig_acc_dist.update_layout(
    title='Distribution of Per-Class Accuracy',
    xaxis_title='Accuracy',
    yaxis_title='Number of Classes',
    showlegend=False,
    height=500
)
fig_acc_dist.show()


Top 10 Best Performing Categories:
                 category  accuracy  test_samples
               faces-easy  0.974684            79
               motorbikes  0.951389           144
                airplanes  0.909722           144
                 car-side  0.904762            21
                 leopards  0.885714            35
                sunflower  0.866667            15
                     mars  0.793103            29
                trilobite  0.764706            17
self-propelled-lawn-mower  0.727273            22
                 hibiscus  0.700000            20

Top 10 Worst Performing Categories:
         category  accuracy  test_samples
           cannon       0.0            19
fire-extinguisher       0.0            16
            blimp       0.0            16
        paperclip       0.0            17
            spoon       0.0            19
              dog       0.0            19
      floppy-disk       0.0            15
         goldfish       0.0            17

![Distribution of Per-Class Accuracy](images/6-3.png)

| Best Categories | Accuracy | Worst Categories  | Accuracy |
| --------------- | -------- | ----------------- | -------- |
| faces-easy      | 97.5%    | cannon            | 0%       |
| motorbikes      | 95.1%    | fire-extinguisher | 0%       |
| airplanes       | 90.9%    | blimp             | 0%       |
| car-side        | 90.5%    | paperclip         | 0%       |
| leopards        | 88.6%    | spoon             | 0%       |

- Mean accuracy: 25.32%
- Median accuracy: 22.22%
- Standard deviation: 20.26%

The histogram shows a right-skewed distribution: most categories cluster around 10-25% accuracy, while a small subset achieves 70%+ accuracy. Categories that perform well (faces, motorbikes, airplanes) have distinctive visual signatures—consistent shapes, textures, or compositions that make them stand out. Categories that fail completely (0% accuracy) tend to be small objects (paperclip, spoon) or items that appear in varied contexts (dog, refrigerator), making them visually ambiguous.

The high standard deviation (20%) confirms that performance varies dramatically across categories: some are easily separable while others are essentially unrecognizable to the model. This reinforces our earlier finding: visual features work well for visually distinctive categories but struggle with generic or context-dependent objects.